In [1]:
import pandas as pd
import re
import sys

In [3]:
# def split_test_methods(df, column_name):
#     df = df.copy()
#     for index, row in df.iterrows():
#         test_class_def = row[column_name]
#         class_name = re.search(r'class (\w+)', test_class_def).group(1)
#         test_methods = re.findall(r'def (\w+)', test_class_def)

#         new_method_defs = ""
#         for method in test_methods:
#             pattern = r'def ' + method + r'\(self\):([\s\S]*?)(?=def \w+\(self\)|$)'
#             method_body = re.search(pattern, test_class_def)
#             try:
#                 method_body = method_body.group(1)
#             except:
#                 print(method)
#                 print(index, row)
#                 sys.exit()
#             asserts = re.findall(r'(self\.assert\w+\(.*?\))', method_body)
#             print(asserts)
#             if len(asserts) > 1:
#                 for i, assert_stmt in enumerate(asserts, 1):
#                     new_method_name = f"{method}_{i}"
#                     new_method_def = f"    def {new_method_name}(self):\n        {assert_stmt}\n"
#                     new_method_defs += new_method_def
#             else:
#                 # Mantendo a formatação original para métodos com um único assert
#                 method_body_corrected = method_body.strip()
#                 new_method_defs += f"    def {method}(self):\n"+ "        " + f"{method_body_corrected}\n\n"

#         df.at[index, column_name] = f"class {class_name}:\n{new_method_defs.rstrip()}"

#     return df

# Função Para Separar Múltiplos Assert 

In [4]:
def split_test_methods(df, column_name):
    df = df.copy()
    for index, row in df.iterrows():
        test_class_def = row[column_name]
        class_name = re.search(r'class (\w+)', test_class_def).group(1)
        test_methods = re.findall(r'def (\w+)', test_class_def)

        new_method_defs = ""
        for method in test_methods:
            pattern = r'def ' + method + r'\(self\):([\s\S]*?)(?=def \w+\(self\)|$)'
            method_body = re.search(pattern, test_class_def)
            try:
                method_body = method_body.group(1)
            except:
                print(method)
                print(index, row)
                sys.exit()
            asserts = re.findall(r'(self\.assert\w+\(.*\))', method_body)
            if len(asserts) > 1:
                for i, assert_stmt in enumerate(asserts, 1):
                    new_method_name = f"{method}_{i}"
                    new_method_def = f"    def {new_method_name}(self):\n        {assert_stmt}\n"
                    new_method_defs += new_method_def
            else:
                # Mantendo a formatação original para métodos com um único assert
                method_body_corrected = method_body.strip()
                new_method_defs += f"    def {method}(self):\n"+ "        " + f"{method_body_corrected}\n\n"

        df.at[index, column_name] = f"class {class_name}:\n{new_method_defs.rstrip()}"

    return df

# Leitura e Execução 

In [5]:
df = pd.read_csv(
    '../../1_INFERANCE_UNIT_TEST/CodeLLaMA-7B/generated_tests_codellama7b_replicate.csv',
    sep=',',
    header=0
)

In [6]:
df = split_test_methods(df, 'generated_solution')

In [7]:
df.to_csv('generated_tests_codellama7b_replicate_split_adjusted.csv', index=False)

In [8]:
df.shape

(164, 3)